In [1]:
%matplotlib inline
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, initializers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
file_paths = glob.glob(".\\data\\*")
print(file_paths)

['.\\data\\20200319_not_seating.csv', '.\\data\\20200319_seating.csv', '.\\data\\20200322a_not_seating.csv', '.\\data\\20200323a_seating.csv', '.\\data\\20200323b_not_seating.csv', '.\\data\\20200323c_not_seating.csv', '.\\data\\20200324a_seating.csv', '.\\data\\20200324b_seating.csv', '.\\data\\20200324c_not_seating.csv', '.\\data\\20200324d_not_seating.csv', '.\\data\\20200324f_seating.csv', '.\\data\\20200325a_seating.csv', '.\\data\\20200325b_seating.csv', '.\\data\\20200325c_seating.csv', '.\\data\\20200325e_not_seating.csv', '.\\data\\20200326a_seating.csv', '.\\data\\20200326b_not_seating.csv']


In [3]:
category =np.empty((0,1), float)
rssi =np.empty((0,100), float)

for file in file_paths:
    d = np.loadtxt(file, delimiter=',')
    category_tmp, rssi_tmp = np.hsplit(d, [1])
    rssi = np.concatenate([rssi, rssi_tmp], axis=0)
    category = np.concatenate([category, category_tmp], axis=0)

rssi = rssi * (-1) / 128
print("rssi array shape : ", rssi.shape)
#print(rssi)
category = tf.keras.utils.to_categorical(category, 2)
print("category array shape : ", category.shape)
#print(category)

rssi_train, rssi_test, category_train, category_test = train_test_split(rssi, category, test_size=0.3)
print("rssi training array shape : ", rssi_train.shape)
#print(rssi_train)
print("category training array shape : ", category_train.shape)
#print(category_train)
print("rssi test array shape : ", rssi_test.shape)
print("category test array shape : ", category_test.shape)

#train_data = tf.data.Dataset.from_tensor_slices((rssi_train, category_train))
#print(train_data)

rssi array shape :  (7297, 100)
category array shape :  (7297, 2)
rssi training array shape :  (5107, 100)
category training array shape :  (5107, 2)
rssi test array shape :  (2190, 100)
category test array shape :  (2190, 2)


In [4]:
# モデルを作成
model = models.Sequential()
model.add(layers.Dense(128, input_shape=(100, ), activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
#model.add(layers.Dropout(0.1))
model.add(layers.Dense(2, activation='softmax'))

# サマリーを出力
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               12928     
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 130       
Total params: 87,234
Trainable params: 87,234
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

training = model.fit(rssi_train, category_train,
                    batch_size=128,
                    epochs=256,
                    #verbose=1,
                    validation_data=(rssi_test, category_test))

#正答率
plt.plot(training.history['accuracy'])
plt.plot(training.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#loss
plt.plot(training.history['loss'])
plt.plot(training.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Train on 5107 samples, validate on 2190 samples
Epoch 1/256
5107/5107 [==============================] - 1s 168us/sample - loss: 0.6686 - accuracy: 0.6207 - val_loss: 0.6679 - val_accuracy: 0.6183
Epoch 2/256
5107/5107 [==============================] - 0s 29us/sample - loss: 0.6681 - accuracy: 0.6207 - val_loss: 0.6671 - val_accuracy: 0.6183
Epoch 3/256
5107/5107 [==============================] - 0s 29us/sample - loss: 0.6661 - accuracy: 0.6207 - val_loss: 0.6674 - val_accuracy: 0.6183
Epoch 4/256
5107/5107 [==============================] - 0s 30us/sample - loss: 0.6664 - accuracy: 0.6207 - val_loss: 0.6666 - val_accuracy: 0.6183
Epoch 5/256
5107/5107 [==============================] - 0s 25us/sample - loss: 0.6653 - accuracy: 0.6207 - val_loss: 0.6649 - val_accuracy: 0.6183
Epoch 6/256
5107/5107 [==============================] - 0s 31us/sample - loss: 0.6636 - accuracy: 0.6207 - val_loss: 0.6646 - val_accuracy: 0.6183
Epoch 7/256
5107/5107 [==============================] - 0s 25u

Epoch 56/256
5107/5107 [==============================] - 0s 29us/sample - loss: 0.5745 - accuracy: 0.7270 - val_loss: 0.5832 - val_accuracy: 0.7201
Epoch 57/256
5107/5107 [==============================] - 0s 26us/sample - loss: 0.5736 - accuracy: 0.7282 - val_loss: 0.5828 - val_accuracy: 0.7247
Epoch 58/256
5107/5107 [==============================] - 0s 29us/sample - loss: 0.5724 - accuracy: 0.7294 - val_loss: 0.5841 - val_accuracy: 0.7237
Epoch 59/256
5107/5107 [==============================] - 0s 29us/sample - loss: 0.5673 - accuracy: 0.7337 - val_loss: 0.5859 - val_accuracy: 0.7233
Epoch 60/256
5107/5107 [==============================] - 0s 32us/sample - loss: 0.5717 - accuracy: 0.7290 - val_loss: 0.5814 - val_accuracy: 0.7228
Epoch 61/256
5107/5107 [==============================] - 0s 35us/sample - loss: 0.5652 - accuracy: 0.7366 - val_loss: 0.5836 - val_accuracy: 0.7260
Epoch 62/256
5107/5107 [==============================] - 0s 28us/sample - loss: 0.5682 - accuracy: 0.7337

In [6]:
model.save('model/seating_detection_algorithm.h5', save_format='h5')

In [7]:
import tensorflow as tf
from tensorflow.keras import models
import numpy as np

model = models.load_model('model/seating_detection_algorithm.h5')

x = np.loadtxt(".\data\\20200319_seating.csv", delimiter=',')
#print("x array shape : ", x.shape)
#print(x)

x = x[0]
#print("x array shape : ", x.shape)
#print(x)
x = np.delete(x, 0)
#print("x array shape : ", x.shape)
#print(x)
x = x.reshape(1,100)
#print("x array shape : ", x.shape)
#print(x)
x = x * (-1) / 128
#print("x array shape : ", x.shape)
#print(x)

print(np.argmax(model.predict(x)))
#l = model.predict(x)

1
